훈련하는 도중이나 훈련이 끝난 후에 모델을 저장할 수 있습니다. 모델을 중지된 지점부터 다시 훈련할 수 있어 한 번에 오랫동안 훈련하지 않아도 됩니다. 또 모델을 저장하면 다른 사람에게 공유할 수 있고 작업을 재현할 수 있습니다. 연구한 모델과 기법을 공개할 때 많은 머신 러닝 기술자들이 다음과 같은 것들을 제공합니다:
* 모델을 만드는 코드
* 모델의 훈련된 가중치 또는 파라미터

## 저장 방식
사용하는 API에 따라서 여러가지 방법으로 텐서플로 모델을 저장할 수 있습니다. 이 문서는 텐서플로 모델을 만들고 훈련하기 위한 고수준 API인 tf.keras를 사용합니다. 다른 방법들에 대해서는 텐서플로의 저장과 복원 문서와 즉시 실행(eager execution) 문서의 저장하기 섹션을 참고하세요.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0'

In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

## 모델 정의

In [5]:
# 간단한 Sequential 모델을 반환합니다
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model


# 모델 객체를 만듭니다
model = create_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 학습동안 체크포인트 저장하기
훈련 중간과 훈련 마지막에 체크포인트(checkpoint)를 자동으로 저장하도록 하는 것이 많이 사용하는 방법입니다. 다시 훈련하지 않고 모델을 재사용하거나 훈련 과정이 중지된 경우 이어서 훈련을 진행할 수 있습니다.

`tf.keras.callbacks.ModelCheckpoint`은 이런 작업을 수행하는 콜백(callback)입니다. 이 콜백은 체크포인트 작업을 조정할 수 있도록 여러가지 매개변수를 제공합니다.

In [6]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)

model = create_model()

model.fit(train_images, train_labels, epochs = 10,
         validation_data = (test_images, test_labels),
         callbacks = [cp_callback])  # 훈련 단계에서 콜백 전달
# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 832/1000 [=======================>......] - ETA: 0s - loss: 1.2665 - accuracy: 0.6370 
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 1ms/sample - loss: 1.1601 - accuracy: 0.6650 - val_loss: 0.7211 - val_accuracy: 0.7810
Epoch 2/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.4454 - accuracy: 0.8798
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 348us/sample - loss: 0.4408 - accuracy: 0.8790 - val_loss: 0.5389 - val_accuracy: 0.8450
Epoch 3/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.3045 - accuracy: 0.9125
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 390us/sample - loss: 0.2958 - accuracy: 0.9170 - val_loss: 0.4866 - val_accuracy: 0.8500
Epoch 4/10
 704/1000 [====================>.........] - ETA: 0s - loss: 0.2243 - accuracy: 0.9403
Epoch 

In [8]:
!ls {checkpoint_dir}

checkpoint
cp.ckpt.data-00000-of-00002
cp.ckpt.data-00001-of-00002
cp.ckpt.index


훈련하지 않은 모델을 이용한 정확도

In [9]:
model = create_model()

loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 2.3147 - accuracy: 0.1040
훈련되지 않은 모델의 정확도: 10.40%


체크포인트에서 가중치를 로드하고 다시 평가

In [10]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4642 - accuracy: 0.8700
복원된 모델의 정확도: 87.00%


### 체크포인트 콜백 매개변수
이 콜백 함수는 몇 가지 매개변수를 제공합니다. 체크포인트 이름을 고유하게 만들거나 체크포인트 주기를 조정할 수 있습니다.

새로운 모델을 훈련하고 다섯 번의 에포크마다 고유한 이름으로 체크포인트를 저장해 보겠습니다:

In [11]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 다섯 번째 에포크마다 가중치를 저장합니다
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [12]:
! ls {checkpoint_dir}

checkpoint
cp-0000.ckpt.data-00000-of-00002
cp-0000.ckpt.data-00001-of-00002
cp-0000.ckpt.index
cp-0005.ckpt.data-00000-of-00002
cp-0005.ckpt.data-00001-of-00002
cp-0005.ckpt.index
cp-0010.ckpt.data-00000-of-00002
cp-0010.ckpt.data-00001-of-00002
cp-0010.ckpt.index
cp-0015.ckpt.data-00000-of-00002
cp-0015.ckpt.data-00001-of-00002
cp-0015.ckpt.index
cp-0020.ckpt.data-00000-of-00002
cp-0020.ckpt.data-00001-of-00002
cp-0020.ckpt.index
cp-0025.ckpt.data-00000-of-00002
cp-0025.ckpt.data-00001-of-00002
cp-0025.ckpt.index
cp-0030.ckpt.data-00000-of-00002
cp-0030.ckpt.data-00001-of-00002
cp-0030.ckpt.index
cp-0035.ckpt.data-00000-of-00002
cp-0035.ckpt.data-00001-of-00002
cp-0035.ckpt.index
cp-0040.ckpt.data-00000-of-00002
cp-0040.ckpt.data-00001-of-00002
cp-0040.ckpt.index
cp-0045.ckpt.data-00000-of-00002
cp-0045.ckpt.data-00001-of-00002
cp-0045.ckpt.index
cp-0050.ckpt.data-00000-of-00002
cp-0050.ckpt.data-00001-of-00002
cp-0050.ckpt.index


In [13]:
# 가장 최근에 저장한 checkpoint path
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [17]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.5130 - accuracy: 0.8820
복원된 모델의 정확도: 88.20%


## 수동으로 가중치 저장하기

In [19]:
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 가중치를 복원합니다
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.5130 - accuracy: 0.8820
복원된 모델의 정확도: 88.20%


## 모델 전체 저장
전체 모델을 파일 하나에 저장할 수 있습니다. 여기에는 가중치, 모델 구성 심지어 옵티마이저에 지정한 설정까지 포함됩니다. 모델의 체크포인트를 저장하므로 원본 코드를 사용하지 않고 나중에 정확히 동일한 상태에서 훈련을 다시 시작할 수 있습니다.

전체 모델을 저장하는 기능은 매우 유용합니다. TensorFlow.js로 모델을 로드한 다음 웹 브라우저에서 모델을 훈련하고 실행할 수 있습니다(HDF5, Saved Model). 또는 모바일 장치에 맞도록 변환한 다음 TensorFlow Lite를 사용하여 실행할 수 있습니다(HDF5, Saved Model).

### HDF5 파일로 저장하기

In [20]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 308us/sample - loss: 1.1576 - accuracy: 0.6700
Epoch 2/5
1000/1000 [==============================] - 0s 64us/sample - loss: 0.4340 - accuracy: 0.8850
Epoch 3/5
1000/1000 [==============================] - 0s 69us/sample - loss: 0.2976 - accuracy: 0.9170
Epoch 4/5
1000/1000 [==============================] - 0s 74us/sample - loss: 0.2116 - accuracy: 0.9520
Epoch 5/5
1000/1000 [==============================] - 0s 67us/sample - loss: 0.1559 - accuracy: 0.9690


In [21]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [22]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4443 - accuracy: 0.8510
복원된 모델의 정확도: 85.10%


이 방법은 모든 것을 저장한다.
* 가중치
* 모델 설정
* 옵티마이저 설정

In [32]:
new_model.optimizer

## saved_model 사용

In [33]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 305us/sample - loss: 1.1343 - accuracy: 0.6760
Epoch 2/5
1000/1000 [==============================] - 0s 68us/sample - loss: 0.4240 - accuracy: 0.8770
Epoch 3/5
1000/1000 [==============================] - 0s 67us/sample - loss: 0.2917 - accuracy: 0.9290
Epoch 4/5
1000/1000 [==============================] - 0s 67us/sample - loss: 0.2036 - accuracy: 0.9530
Epoch 5/5
1000/1000 [==============================] - 0s 68us/sample - loss: 0.1585 - accuracy: 0.9630


In [34]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO

'./saved_models/1582785290'

In [35]:
!ls saved_models/

1582785290


In [36]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()

Instructions for updating:
The experimental save and load functions have been  deprecated. Please switch to `tf.keras.models.load_model`.
Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.predict(test_images).shape

(1000, 10)

load된 모델 평가를 위해서는 컴파일이 필요

In [38]:
# 이 모델을 평가하려면 그전에 컴파일해야 합니다.
# 단지 저장된 모델의 배포라면 이 단계가 필요하지 않습니다.

new_model.compile(optimizer=model.optimizer, # 복원된 옵티마이저를 사용합니다.
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4653 - accuracy: 0.8670
복원된 모델의 정확도: 86.70%
